In [1]:
import markdown
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime as dt
import sqlalchemy 
import psycopg2
import os
import sys

In [2]:
sys.path.append('..')

In [3]:
path='https://raw.githubusercontent.com/panchohumeres/dynamo-covid/master/fuentes.md'
req=requests.get(path)
#files=req.text.split(files_folder_prefix)

In [4]:
page = BeautifulSoup(markdown.markdown(req.text), "html")
fuentes=page.find_all('li')
descs=[]
idxs=[]
for idx in range(len(fuentes)):
    descs.append(fuentes[idx].get_text().replace('\n',''))
    idxs.append(idx+1)

In [5]:
fuentes=pd.DataFrame({'fuentes':descs,'N°':idxs})

In [6]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [7]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [8]:
for k in ['fuentes']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema fuentes exists, will not be created


In [9]:
schema='fuentes'
name='fuentes'
print("creating table "+name+' ,schema: '+schema)
fuentes.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
fuentes.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table fuentes ,schema: fuentes
saving table/data/ETLcache/fuentes_01022021_033217.csv in cache
